In [96]:
using JuMP, Gurobi, DataFrames, CSV, Random, LinearAlgebra, PyPlot, Dates, Statistics

In [126]:
gurobi_env = Gurobi.Env()

Academic license - for non-commercial use only


Gurobi.Env(Ptr{Nothing} @0x00007ff0c089d800)

This function is a helper function to compute the g(z)

In [130]:
function g(y, X, z, λ)
    model = Model(solver = GurobiSolver(TimeLimit = 60, OutputFlag = 0, gurobi_env))
    
    @variable(model, μ[1:n, 1:n] >= 0 )
    
    @objective(model, Max, 
        -1/2 * sum((y[i] + sum(μ[j, i] for j=1:n) + sum(μ[i, j] for j=1:n))^2 for i=1:n)
        - 1/(2*λ) * sum(sum(z[l]*(sum(μ[i, j]*(X[j, l] - X[i, l]) for j=1:n))^2 for l=1:d) for i = 1:n))
    
    sol = solve(model)
    return (getobjectivevalue(model), getvalue(μ))
    
end

g (generic function with 1 method)

This function is a helper function to compute the subgradient of g

In [124]:
function subgradient_g(y, X, z, λ)
    obj, μ = g(y, X, z, λ)
    n,d = size(X)
    subdradient = zeros(d)
    for p in 1:d
        outer_sum = 0
        inner_sum = zeros(n)
        for i in 1:n
            for j in 1:n
                inner_sum[i] += μ[i, j]*(X[i,p] - X[j, p])
            end
            outer_sum += inner_sum[i]^2
            subdradient[p] = -1/(2*λ) * outer_sum
        end
    end
    return(subdradient)
end 

subgradient_g (generic function with 1 method)

In [129]:
function cutting_plane(λ, ϵ, X, y, k)
    n,d = size(X)
    γ_t, g_t, t = 0, 0, 0
    z_t = zeros(d)
    
    model = Model(solver = GurobiSolver(TimeLimit = 60, OutputFlag = 0, gurobi_env))
    @variable(model, z[1:d], Bin)
    @variable(model, γ)
    @objective(model, Min,  γ)
    @constraint(model, sum(z[i] for i=1:d) <= k)

    while γ_t < g_t + ϵ
        # Add a new constraint
        subgradient = convert(Matrix, subgradient_g(y, X, z_t, λ)')
        @constraint(model, g(y, X, z_t, λ)[1] + sum(subgradient[i]*(z[i] - z_t[i]) for i= 1:d) <= γ)
        
        # Resolve the optimization problem
        sol = solve(model)
        γ_t = getobjectivevalue(model)
        z_t = getvalue(z)
        t += 1
    end
    return (z)
end

cutting_plane (generic function with 1 method)

## Autres essais

In [127]:
X = [1 1 1; 2 2 2; 3 3 3 ; 4 4 4]
y = [1 2 3 4]
k = 2
λ = 1
ϵ = 1

BoundsError: BoundsError: attempt to access 2-element Array{Int64,1} at index [3]

In [131]:
X = [1 1 1; 2 2 2; 4 3 5 ; 5 2 6]
y = [3 5 6 7]
k = 2
λ = 1
ϵ = 0.1
n,d = size(X)
z_t = zeros(d)

c = convert(Matrix, subgradient_g(y, X, z_t, λ)')


1×3 Array{Float64,2}:
 -0.0  -0.0  -0.0

In [86]:
function regularized_problem(λ, X, y, k, g_z, dg_z, z_t)
    n,d = size(X)
    model = Model(solver = GurobiSolver(TimeLimit = 60, OutputFlag = 0))
    
    @variable(model, z[1:d], Bin)
    @variable(model, γ)
    
    @objective(model, Min,  γ)
    
    @constraint(model, sum(z[i] for i=1:d) <= k)
    
    for i in 1:size(g_z)
        @constraint(model, g_z[i] + dg_z[i,:]'*(z - z_t[i,:]) <= γ)
    end
        
    sol = solve(model)
    return (getobjectivevalue(model), getvalue(z))
end

regularized_problem (generic function with 1 method)

In [87]:
function cutting_plane(λ, ϵ, X, y, k)
    n,d = size(X)
    γ_t, g_t, t = 0, 0, 0
    z_t = zeros(d)
    
    g_constraints = []
    dg_constraints = []
    z_constraints = []
    
    while γ < gt + ϵ
        # Add a new constraint
        append!(g_z, g(y, X, z_t, λ)[1])
        append!(z_t, z)
        append!(dg_z, subgradient_g(y, X, z, λ))
        
        # Resolve the optimization problem
        γ_t, z_t = regularized_problem(λ, X, y, k, g_constraints, dg_constraints, z_constraints)
        t += 1
    end
    return (z)
end
    

cutting_plane (generic function with 1 method)